FUNCTION
===================

TIME AND DATE
===================

INPUT(): "reread" a variable according to a new informat, char to numeric (IN for INteger)  
take the first argument and "reads" it as if it were coming from a file, according to the INFORMAT listed as the second argument.  

PUT: use formatted value of a variable to create new variable, numeric to char 
it "writes out" the value of the first argument, according to the FORMAT in second argument. By "write out" we actually mean assign the value to the variable on the left of the equal sign.  

STRING
=========
concantenate: `||`

COMPBL(var) # compress multi-blank into single blank
COMPRESS(VAR,'charToRm')
SUBSTR(char_var,start,length)
INDEX('ABCDEFG','DEF'); #return 4
LENGTH() #trailing blacks are excluded in computation;
VERIFY(ANSWER, 'ABCDE'); # make sure answer has only ABCDE
SCAN(char_var,n,'list-of-delimiters'); #SCAN("a-b,c;d" ,2,'-,;'); RETURN b

In [1]:

DATA EX_10; 
   INPUT LONG_STR $ 1-80;
   ARRAY PIECES[5] $ 10   
      PIECE1-PIECE5;
   DO I = 1 TO 5;
      PIECES[I] = SCAN(LONG_STR,I,',;.! ');
   END;
   DROP LONG_STR I;
DATALINES4;
THIS LINE,CONTAINS!FIVE.WORDS
ABCDEFGHIJKL XXX;YYY
;;;;
PROC PRINT DATA=EX_10 NOOBS;
   TITLE 'Listing of Example 10';
RUN;

PIECE1,PIECE2,PIECE3,PIECE4,PIECE5
THIS,LINE,CONTAINS,FIVE,WORDS
ABCDEFGHIJ,XXX,YYY,,


In [2]:

DATA EX_12;
   LENGTH A B C D E $ 1;
   INPUT A B C D E X Y;
DATALINES;
M f P p D 1 2
m f m F M 3 4
;
DATA UPPER;
   SET EX_12;
   ARRAY ALL_C[*] _CHARACTER_;
   DO I = 1 TO DIM(ALL_C);
      ALL_C[I] = UPCASE(ALL_C[I]);
   END;
   DROP I;
RUN;

PROC PRINT DATA=UPPER NOOBS;
   TITLE 'Listing of UPPER';
RUN;

A,B,C,D,E,X,Y
M,F,P,P,D,1,2
M,F,M,F,M,3,4


In [6]:

DATA EX_13;
   INPUT QUES : $1. @@;
   QUES = TRANSLATE(QUES,'ABCDE','12345');
DATALINES;
1 4 3 2 5
5 3 4 2 1
;
PROC PRINT DATA=EX_13(obs=3) NOOBS;
   TITLE 'LISTING OF EXAMPLE 13';
RUN;


QUES
A
D
C


In [7]:
DATA CONVERT;
   INPUT @1 ADDRESS $20. ;
   *** Convert Street, Avenue, and Boulevard to their abbreviations;
   ADDRESS = TRANWRD (ADDRESS,'Street','St.');
   ADDRESS = TRANWRD (ADDRESS,'Avenue','Ave.');
   ADDRESS = TRANWRD (ADDRESS,'Road','Rd.');
DATALINES;
89 Lazy Brook Road 
123 River Rd.
12 Main Street
;
PROC PRINT DATA=CONVERT;
   TITLE 'Listing of Data Set CONVERT';
RUN;

Obs,ADDRESS
1,89 Lazy Brook Rd.
2,123 River Rd.
3,12 Main St.


LAG & DIFF
=============
LAGn()  
DIFn()

TEMPLATE
============

Values as % of Grand Mean
--------------------------------
Use '_N_' to conditionally exec. SET stmt, any variable coming in from a SET stmt are automatically retained so the values of MHR, MSBP, MDBP will not change and will be avilable for every iteration of DATA STEP.

In [9]:
DATA TEST;
   INPUT HR SBP DBP;
DATALINES;
80 160 100
70 150 90
60 140 80
;
PROC MEANS NOPRINT DATA=TEST;
   VAR HR SBP DBP;
   OUTPUT OUT=MOUT(DROP=_TYPE_ _FREQ_)
          MEAN=MHR MSBP MDBP;
RUN;

DATA NEW (DROP=MHR MSBP MDBP);
   SET TEST;
   IF _N_ = 1 THEN SET MOUT;
   HRPER=100*HR/MHR;
   SBPPER=100*SBP/MSBP;
   DBPPER=100*DBP/MDBP;
RUN;

PROC PRINT DATA=NEW NOOBS;
   TITLE 'Listing of Data Set NEW';
RUN;      

HR,SBP,DBP,HRPER,SBPPER,DBPPER
80,160,100,114.286,106.667,111.111
70,150,90,100.000,100.000,100.000
60,140,80,85.714,93.333,88.889


Value as % of Group Mean
------------------------------------

In [10]:

DATA TEST; 
   INPUT GROUP $ HR SBP DBP @@; 
DATALINES; 
A 80 160 100 A 70 150 90 A 60 140 80 
B 90 200 180 B 80 180 140 B 70 140 80 
; 
PROC SORT DATA=TEST; 
   BY GROUP;
RUN;

PROC MEANS DATA=TEST NOPRINT NWAY;
   CLASS GROUP;
   VAR HR SBP DBP;
   OUTPUT OUT=MOUT(DROP=_TYPE_ _FREQ_)
          MEAN=MHR MSBP MDBP;
RUN;

DATA NEW (DROP=MHR MSBP MDBP);
   MERGE TEST MOUT;
      BY GROUP;
   HRPER=100*HR/MHR;
   SBPPER=100*SBP/MSBP;
   DBPPER=100*DBP/MDBP;
RUN;

PROC PRINT NOOBS;
   TITLE 'Listing of Data Set NEW';
RUN;                                          


GROUP,HR,SBP,DBP,HRPER,SBPPER,DBPPER
A,80,160,100,114.286,106.667,111.111
A,70,150,90,100.000,100.000,100.000
A,60,140,80,85.714,93.333,88.889
B,90,200,180,112.500,115.385,135.000
B,80,180,140,100.000,103.846,105.000
B,70,140,80,87.500,80.769,60.000


Plotting Means with Error Bars
---------------------------------

In [11]:
ATA ORIG;
  INPUT SUBJ TIME DBP SBP;
DATALINES;
1 1 70 120
1 2 80 130
1 3 84 136
2 1 82 132
2 2 84 138
2 3 92 144
;
PROC MEANS DATA=ORIG NOPRINT NWAY;
   CLASS TIME;
   VAR SBP DBP;
   OUTPUT OUT=MEANOUT MEAN= STDERR=SE_SBP SE_DBP;
RUN;

DATA TMP;
   SET MEANOUT;
   SBPHI=SBP+SE_SBP;
   SBPLO=SBP-SE_SBP;
   DBPHI=DBP+SE_DBP;
   DBPLO=DBP-SE_DBP;
RUN;

PROC PLOT DATA=TMP;
   PLOT SBP*TIME='o' SBPHI*TIME='-' SBPLO*TIME='-' / OVERLAY BOX;
   PLOT DBP*TIME='o' DBPHI*TIME='-' DBPLO*TIME='-' / OVERLAY BOX;
   TITLE 'Plot of Mean Blood Pressures at Each Time';
   TITLE2 'Error bars represent +- 1 standard error';
RUN;a